# **Procesamiento de Lenguaje Natural**

## Maestría en Inteligencia Artificial Aplicada  
### Tecnológico de Monterrey  
### Prof Luis Eduardo Falcón Morales  

### **Actividad en Equipos: sistema LLM + RAG**


* **Nombres y matrículas:**

  * Jorge Pasalagua Sosa - **A01793896**
  * Luis Angel Herrera Flores - **A01796156**
  * Mario Raúl García Ramírez - **A01658682**
  * Oscar Luis Guadarrama Jiménez - **A01796245**
* **Número de Equipo:** 61


## **Introducción de la problemática a resolver**

En esta actividad se implementa un chatbot basado en un modelo de lenguaje de gran tamaño (LLM) con un sistema de recuperación aumentada generativa (RAG), enfocado en facilitar el entendimiento y consulta de la normatividad aplicable en materia de competencia económica en México, con énfasis en los sectores de telecomunicaciones y radiodifusión.

El chatbot permite realizar preguntas en lenguaje natural y obtener respuestas contextualizadas con base en la Constitución, la Ley Federal de Competencia Económica, la Ley Federal de Telecomunicaciones y Radiodifusión, guías del Instituto Federal de TElecomunicaciones (IFT) y opiniones públicas relevantes.

Esta herramienta busca apoyar a analistas, abogados y personal del IFT en la interpretación y aplicación de la normatividad vigente, mejorando la eficiencia y precisión en el trabajo diario.

*Nota:* El marco normativo en esta materia se encuentra sujeto a posibles reformas. Por lo tanto, una herramienta como la que se plantea en estre traabjao deberá prever mecanismos de actualización periódica para reflejar con fidelidad el marco normativo vigente en cada momento.

## **Sistema RAG + LLM**

### **Arquitectura**

- **LLM:** GPT-4o vía API de OpenAI.
- **RAG:** Sistema de recuperación basado en embeddings de documentos legales.
- **Vector Store:** ChromaDB.
- **Embeddings:** `OpenAIEmbeddings`.
- **Carga de documentos:** Archivos PDF procesados con `pdfplumber`.

### **Pipeline**

1. Conversión de documentos PDF a texto.
2. Segmentación de texto en fragmentos manejables.
3. Creación del índice de vectores (embeddings).
4. Recuperación de fragmentos relevantes según la pregunta del usuario.
5. Generación de respuesta con GPT-4o, usando el contexto recuperado.


In [48]:
import os
import pdfplumber
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from dotenv import load_dotenv
import openai
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI
from langchain.callbacks import get_openai_callback
from datetime import datetime
import gradio as gr

# **Carga de documentos PDF con metadatos**

Esta sección tiene como objetivo cargar todos los documentos normativos relevantes desde una carpeta local, extraer su contenido textual y asociar cada fragmento con el nombre del archivo original como metadato. Esto permite:

- Rastrear la fuente exacta de cada respuesta generada por el chatbot.
- Identificar fácilmente a qué documento pertenece cada fragmento dentro del vectorstore.
- Permitir búsquedas y referencias más precisas durante la consulta legal.

El corpus documental incluye:

- Constitución Política de los Estados Unidos Mexicanos
- Ley Federal de Competencia Económica (LFCE)
- Ley Federal de Telecomunicaciones y Radiodifusión (LFTR)
- Disposiciones regulatorias emitidas por el IFT
- Guías, criterios técnicos y resoluciones públicas

Este paso es fundamental para asegurar la **trazabilidad jurídica** y la **confiabilidad normativa** de las respuestas proporcionadas por el modelo.


In [8]:
def cargar_documentos_con_metadatos(carpeta):
    documentos = []
    for archivo in os.listdir(carpeta):
        if archivo.endswith(".pdf"):
            ruta = os.path.join(carpeta, archivo)
            with pdfplumber.open(ruta) as pdf:
                texto = ""
                for pagina in pdf.pages:
                    contenido = pagina.extract_text()
                    if contenido:
                        texto += contenido + "\n"
                documentos.append({"filename": archivo, "text": texto})
    return documentos

# Ejecutar con la carpeta que contiene tus PDFs
documentos = cargar_documentos_con_metadatos("documentos")

# Verificación de carga
print(f"Se cargaron {len(documentos)} documentos.")
print(f"Ejemplo: {documentos[0]['filename']}")
print(f"Fragmento:\n{documentos[0]['text'][:300]}")


CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, def

Se cargaron 15 documentos.
Ejemplo: 112_180518.pdf
Fragmento:
LEY FEDERAL DE PROCEDIMIENTO ADMINISTRATIVO
CÁMARA DE DIPUTADOS DEL H. CONGRESO DE LA UNIÓN Última Reforma DOF 18-05-2018
Secretaría General
Secretaría de Servicios Parlamentarios
LEY FEDERAL DE PROCEDIMIENTO ADMINISTRATIVO
Nueva Ley publicada en el Diario Oficial de la Federación el 4 de agosto de 


# **División del texto en fragmentos e inclusión de metadatos**



In [22]:
def preparar_chunks(documentos):
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200
    )
    chunks = []
    for doc in documentos:
        partes = splitter.split_text(doc["text"])
        for parte in partes:
            chunks.append(Document(
                page_content=parte,
                metadata={"source": doc["filename"]}
            ))
    return chunks

# Dividir todos los documentos en fragmentos con metadatos
docs_chunks = preparar_chunks(documentos)

# Verificación
print(f"Total de fragmentos generados: {len(docs_chunks)}")
print("Ejemplo de metadatos:", docs_chunks[0].metadata)
print("Contenido del fragmento:\n", docs_chunks[0].page_content[:300])


Total de fragmentos generados: 5290
Ejemplo de metadatos: {'source': '112_180518.pdf'}
Contenido del fragmento:
 LEY FEDERAL DE PROCEDIMIENTO ADMINISTRATIVO
CÁMARA DE DIPUTADOS DEL H. CONGRESO DE LA UNIÓN Última Reforma DOF 18-05-2018
Secretaría General
Secretaría de Servicios Parlamentarios
LEY FEDERAL DE PROCEDIMIENTO ADMINISTRATIVO
Nueva Ley publicada en el Diario Oficial de la Federación el 4 de agosto de 


# **Creación y persistencia del Vector Store con Chroma**

Esta sección tiene como finalidad transformar los fragmentos de texto legales previamente extraídos en representaciones vectoriales (embeddings) utilizando el modelo `OpenAIEmbeddings`. Estas representaciones permiten realizar búsquedas semánticas más inteligentes y relevantes, incluso cuando el usuario no utiliza exactamente las mismas palabras que aparecen en el documento original.

Posteriormente, los vectores generados se almacenan en una base de datos vectorial llamada `Chroma`, la cual permite consultar y recuperar rápidamente los fragmentos más similares a una pregunta dada. Además, el índice se guarda de forma persistente en disco, lo que evita tener que recalcularlo cada vez que se reinicie el sistema.


In [26]:
# Cargar las variables del archivo .env
load_dotenv()

# Obtener la clave
api_key = os.getenv("OPENAI_API_KEY")

# Configurar la clave para openai (opcional, para llamadas directas con `openai`)
openai.api_key = api_key

# Inicializar modelo de embeddings con la clave
embedding_model = OpenAIEmbeddings(openai_api_key=api_key)

# Crear vectorstore de los chunks con metadatos
vectorstore = Chroma.from_documents(
    docs_chunks,
    embedding_model,
    persist_directory="db"
)

# Guardar el índice
vectorstore.persist()

# Para reutilizar después:
# vectorstore = Chroma(persist_directory="db", embedding_function=embedding_model)

# Definir retriever
retriever = vectorstore.as_retriever(search_kwargs={"k": 4})

print("Vector store creado y guardado exitosamente.")

Vector store creado y guardado exitosamente.


C:\Users\Oscar\AppData\Local\Temp\ipykernel_13484\1363445327.py:21: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectorstore.persist()


# **Integración del modelo LLM (GPT-4o) y construcción de la cadena RAG**

En esta sección se integra el modelo de lenguaje de gran tamaño `GPT-4o` con el sistema de recuperación semántica para construir una cadena de preguntas y respuestas basada en recuperación aumentada por generación (RAG).

Esto permite que el modelo no solo genere texto de manera autónoma, sino que lo haga apoyándose en fragmentos relevantes previamente encontrados en el corpus legal, lo que mejora la precisión y la coherencia de las respuestas.

La cadena RAG se configura para devolver, junto con la respuesta generada, los documentos fuente utilizados, lo cual permite mantener la trazabilidad legal de cada respuesta.


In [28]:
# Crear el modelo LLM con temperatura 0 (respuestas más precisas)
llm = ChatOpenAI(
    model_name="gpt-4o",
    temperature=0
)

# Construcción de la cadena de preguntas-respuestas con recuperación de contexto
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    return_source_documents=True  # Importante para mostrar de qué documento viene la respuesta
)

print("Cadena RAG (GPT-4o + recuperación semántica) lista.")


Cadena RAG (GPT-4o + recuperación semántica) lista.


# **Función de consulta con retroalimentación**

Esta sección define una función llamada `preguntar_con_feedback` que permite enviar preguntas al modelo y obtener una respuesta generada con base en los documentos legales previamente indexados.

La función realiza las siguientes acciones:

- Muestra la pregunta y la respuesta generada por el modelo.
- Informa la cantidad de tokens consumidos en la operación.
- Calcula el costo estimado en dólares de la consulta utilizando el modelo de OpenAI.
- Devuelve un registro estructurado (log) con la pregunta, respuesta, número de tokens, costo y un sello de tiempo (`timestamp`), útil para análisis posteriores o almacenamiento histórico.

Esta estructura permite evaluar tanto la calidad de las respuestas como el rendimiento económico del modelo en cada consulta.



In [42]:
def preguntar_con_feedback(pregunta):
    with get_openai_callback() as cb:
        respuesta = qa_chain({"query": pregunta})
        
        # Mostrar respuesta
        print(f"\nPregunta: {pregunta}\n")
        print(f"Respuesta:\n{respuesta['result']}\n")
                 
        # Mostrar uso de tokens y costo estimado
        print(f"Tokens usados: {cb.total_tokens}")
        print(f"Costo estimado: ${cb.total_cost:.6f}")

        # Retornar log de la interacción (opcional)
        return {
            "timestamp": datetime.now().isoformat(),
            "pregunta": pregunta,
            "respuesta": respuesta["result"],
            "tokens": cb.total_tokens,
            "costo_usd": cb.total_cost
            
        }


# **Pruebas del sistema con preguntas comunes**

En esta sección se realizan pruebas de funcionamiento del chatbot utilizando un conjunto de preguntas frecuentes relacionadas con la normatividad de competencia económica en México.

Cada pregunta es procesada por la función `preguntar_con_feedback`, lo que permite observar:

- La coherencia y exactitud de las respuestas generadas.
- El costo estimado y tokens consumidos por cada consulta.

Esta etapa sirve como validación práctica del sistema antes de ponerlo a disposición de usuarios finales.


In [ ]:
preguntas = [
    "¿Qué es un agente económico con poder sustancial en México?",
    "¿Qué artículo de la LFCE regula las concentraciones?",
    "¿Qué criterios usa el IFT para evaluar barreras a la entrada?",
    "¿Cuál es el marco legal aplicable a la competencia en servicios de telecomunicaciones?",
    "¿Qué prácticas son consideradas monopólicas absolutas según la LFCE?"
]

# Hacer preguntas y medir tokens
for pregunta in preguntas:
   log = preguntar_con_feedback(pregunta)


Pregunta: ¿Qué es un agente económico con poder sustancial en México?

Respuesta:
Un agente económico con poder sustancial en México es una entidad que tiene la capacidad de influir de manera significativa en un mercado relevante, ya sea en los sectores de radiodifusión o telecomunicaciones, o en otros mercados. Esto se determina de acuerdo con el procedimiento establecido en la Ley Federal de Competencia Económica. Para identificar si un agente tiene poder sustancial, se consideran varios criterios, como el grado de posicionamiento de sus bienes o servicios en el mercado, la falta de acceso a importaciones, la existencia de costos elevados de internación, y los diferenciales de costos que enfrentan los consumidores al cambiar de proveedor. Un agente económico con poder sustancial puede estar sujeto a obligaciones específicas impuestas por el Instituto Federal de Telecomunicaciones para asegurar la competencia en el mercado.

Tokens usados: 1036
Costo estimado: $0.003790

Pregunta: ¿Q

# **Interfaz web con Gradio**

Esta sección implementa una interfaz gráfica simple con Gradio que permite a cualquier usuario interactuar con el chatbot desde un navegador web. La interfaz solicita una pregunta en lenguaje natural y devuelve:

- La respuesta generada por el modelo GPT-4o.
- El número de tokens utilizados y el costo estimado de la consulta.

Esto facilita el uso del sistema por parte de usuarios no técnicos, y permite realizar demostraciones funcionales o integrarlo en otros entornos, como páginas web institucionales o prototipos internos.


In [46]:
def interfaz_gradio():
    def responder(pregunta):
        log = preguntar_con_feedback(pregunta)
        texto = f"**Respuesta:**\n{log['respuesta']}\n"
        texto += f"\n**Tokens usados:** {log['tokens']} | **Costo estimado:** ${log['costo_usd']:.6f}"
        return texto

    gr.Interface(
        fn=responder,
        inputs="text",
        outputs="text",
        title="Asistente Jurídico IFT",
        description="Consulta legal basada en la LFCE, LFTR, Constitución y documentos del IFT."
    ).launch()




In [47]:
 interfaz_gradio()

* Running on local URL:  http://127.0.0.1:7864
* To create a public link, set `share=True` in `launch()`.



Pregunta: ¿Cuál es el marco legal aplicable a la competencia en servicios de telecomunicaciones?

Respuesta:
El marco legal aplicable a la competencia en servicios de telecomunicaciones en México incluye varios elementos clave. En primer lugar, está la Constitución Política de los Estados Unidos Mexicanos, específicamente el artículo 28. Además, la Ley Federal de Competencia Económica (LFCE) y sus disposiciones regulatorias son fundamentales, así como la Ley Federal de Telecomunicaciones y Radiodifusión. Estas leyes establecen las medidas y obligaciones para asegurar la competencia efectiva en el sector, como la desagregación de la red local del agente económico preponderante y la extinción de ciertas obligaciones cuando existan condiciones de competencia.

Tokens usados: 1125
Costo estimado: $0.003765

Pregunta:  "¿Qué prácticas son consideradas monopólicas absolutas según la LFCE?"

Respuesta:
No tengo la información específica sobre qué prácticas son consideradas monopólicas absolu

## **Conclusiones**

- El chatbot desarrollado permite consultar de manera razonable la normatividad en materia de competencia económica en México, con énfasis en los sectores de telecomunicaciones y radiodifusión.

- La integración de un sistema RAG con un LLM como GPT-4o mejora la precisión y relevancia de las respuestas, al permitir que el modelo genere texto con base en los documentos legales específicos cargados en el corpus.

- La calidad y utilidad de las respuestas dependen directamente de la cobertura y calidad del corpus documental. En este ejercicio, la combinación de leyes, guías del IFT y opiniones públicas proporciona una base razonablemente sólida.

- Es importante señalar que el marco normativo en esta materia se encuentra sujeto a posibles reformas legislativas en el corto y mediano plazo. Por lo tanto, un despliegue operativo de una herramienta como el chatbot aquí desarrollado debería contemplar un mecanismo de actualización periódica del corpus documental, para asegurar que las respuestas se mantengan alineadas con la normatividad vigente en cada momento.

- El uso de métricas de uso de tokens y costo permite además evaluar la viabilidad económica de este tipo de soluciones en escenarios de uso intensivo.

- Este enfoque podría ser extendido a otras áreas regulatorias del IFT o de la administración pública, facilitando el acceso a marcos normativos complejos mediante interfaces de lenguaje natural.



# **Fin de la actividad chatbot: LLM + RAG**